In [ ]:
using Pkg
Pkg.activate(joinpath(@__DIR__, "../../"))
using CairoMakie  # Load CairoMakie first
CairoMakie.activate!()  # Ensure CairoMakie is active
using Globtim
using DynamicPolynomials, DataFrames
# Constants and Parameters
const n, a, b = 3, 1, 1
const scale_factor = a / b   # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 3e-4            # Define the tolerance for the L2-norm
f = Rastringin # Objective function

We set the number of samples used to generate the approximant. It is annoying that the error goes up while the degree has increased.

In [ ]:
rand_center = [0.0, 0.0, 0,];
d = 10 # initial degree 
SMPL = 40 # Number of samples
TR = test_input(f, 
                dim = n,
                center=rand_center,
                GN=SMPL, 
                sample_range=scale_factor, 
                degree_max =d+4
                )
pol_cheb = Constructor(TR, d, basis=:chebyshev)
# pol_lege = Constructor(TR, d, basis=:legendre);

@polyvar(x[1:n]); # Define polynomial ring 

Solve the system of partial derivatives using `Msolve`. We should allow for points slightly outside the domain. 

In [ ]:
df_cheb = solve_and_parse(pol_cheb, x, f, TR)
sort!(df_cheb, :z, rev=false)

# Enhanced analysis with new statistics  
using Optim
df_cheb_enhanced, df_min_cheb = analyze_critical_points(f, df_cheb, TR, tol_dist=0.05, verbose=true)

println("\n=== Enhanced Statistics Summary ===")
println("Enhanced DataFrame columns: $(names(df_cheb_enhanced))")
println("Enhanced DataFrame size: $(size(df_cheb_enhanced))")
println("Minimizers DataFrame columns: $(names(df_min_cheb))")
println("Minimizers DataFrame size: $(size(df_min_cheb))")

println("\n=== Sample Enhanced Data ===")
if nrow(df_cheb_enhanced) > 0 && :region_id in names(df_cheb_enhanced)
    println("First 3 rows with enhanced statistics:")
    enhanced_cols = [:x1, :x2, :x3, :z, :region_id, :function_value_cluster, :nearest_neighbor_dist, :gradient_norm]
    available_cols = [col for col in enhanced_cols if col in names(df_cheb_enhanced)]
    println(first(df_cheb_enhanced[!, available_cols], 3))
end

if nrow(df_min_cheb) > 0 && :basin_points in names(df_min_cheb)
    println("\nMinimizers with basin analysis:")
    basin_cols = [:x1, :x2, :x3, :value, :basin_points, :average_convergence_steps, :region_coverage_count, :gradient_norm_at_min]
    available_basin_cols = [col for col in basin_cols if col in names(df_min_cheb)]
    println(df_min_cheb[!, available_basin_cols])
end

# Update df_cheb to use enhanced version for visualization
df_cheb = df_cheb_enhanced

In [ ]:
# using StaticArrays
using GLMakie
GLMakie.activate!()  # Ensure GLMakie is active for 3D plots

In [ ]:
grid = scale_factor * generate_grid(3, 100)  # 3D grid
values = map(f, grid); # Prepare level set data for specific level

In [ ]:
fig = create_level_set_visualization(f, grid, df_cheb, (-30.0, 30.))

In [ ]:
# GLMakie.closeall()